# Draft: MIRI LRS Spectral Extraction

***

## Imports
Describe the libraries we're using here. If there's something unusual, explain what the library is, and why we need it.
- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *astropy.table Table* for creating tidy tables of the data
- *matplotlib.pyplot* for plotting data

In [ ]:
# For style checking
# %load_ext pycodestyle_magic
# %flake8_on

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import numpy as np

import astropy.wcs as wcs
from astropy.visualization import simple_norm
from astropy.io import fits
from astropy.table import Table

## Introduction
Extract 1D spectra from 2D MIRI spectral observations.  Show simple "boxcar" extraction as well as a more sophisticated PSF based "optimal" extraction.  PSF based extraction is currently only appropriate for point sources.

Note: Not clear how to use the JWST pipeline `extract_1d` (quite complex) code.

***

## Download File

In [ ]:
#from astropy.utils.data import download_file
#calfile_url = 'https://stsci.box.com/s/17ami7o8r8x49pvarrzgp99e61n794v3'
#calfile = download_file(calfile_url)
calfile = "det_image_seq5_MIRIMAGE_P750Lexp1_cal.fits"

## File information

The downloaded file is a simulation of a LRS slit observation of for the star BD+60d1753, a flux calibration star.  This file was created with MIRISim.
The simulated exposure was reduced using the JWST pipeline (v0.15.0) through the Detector1 and Spec2 stages.

1. (Primary): This HDU contains meta-data related to the observation and data reduction.
2. (SCI): The calibrated image. Units are MJy/sr.
3. (ERR): Uncertainty image.  Units are MJy/sr.
4. (DQ): Data quality image.
5. (VAR_POISSON): Unc. component 1: Poisson uncertainty image.  Units are (MJy/sr)^2.
6. (VAR_RNOISE): Unc. component 2: Read Noise uncertainty image.  Units are (MJy/sr)^2.
7. (VAR_FLAT): Unc. component 3: Flat Field uncertainty image.  Units are (MJy/sr)^2.
8. (ASDF_METADATA): Metadata.

## Loading data

In [ ]:
# load the data
hdu = fits.open(calfile)
# calibrated image in 2nd HDU
data = hdu[1].data

WCS info (not correct - no wavelength info yet)

In [ ]:
imwcs = wcs.WCS(hdu[1].header, hdu)
print(imwcs)

Basic information about the image.

In [ ]:
print(hdu[1].data.shape)
print(np.mean(hdu[1].data))
print(np.amin(hdu[1].data), np.amax(hdu[1].data))

Display the full 2D image

In [ ]:
norm_data = simple_norm(data, 'sqrt')
plt.figure(figsize=(6, 6))
plt.imshow(data, norm=norm_data, origin="lower")
plt.title("The full image from the MIRI IMAGER detector")

Display the LRS Slit region

In [ ]:
# python does y, x
# transpose to make it display better
data_lrs_reg = np.transpose(data[145:385, 300:354]) 

norm_data = simple_norm(data_lrs_reg, "sqrt")
plt.figure(figsize=(10, 3))
plt.imshow(data_lrs_reg, norm=norm_data, origin="lower")
plt.title("The full MIRI IMAGE detector image")

## Boxcar Extraction

Extract a 1D spectrum using a simple boxcar.  Basically collapse the spectrum in the cross-dispersion direction over a specified number of pixels.

Limitation: not clear how to handle bad pixels with boxcar extraction.  Interpolate?

In [ ]:
# for reference read in the JWST pipeline extracted spectrum
# currently the flux column is all zeros - so not that useful
jpipe_1xd = Table.read("det_image_seq5_MIRIMAGE_P750Lexp1_x1d.fits", hdu=1)
# print(jpipe_1xd)

### Fixed width boxcar

Define extraction parameters

In [ ]:
ext_center = 33
ext_width = 8 # in pixels

Plot cross-disperion cut showing the extraction parameters

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
y = np.arange(data_lrs_reg.shape[0])
ax.plot(y, data_lrs_reg[:,100], 'k-')
mm = np.array([ext_center, ext_center])
mm_y = ax.get_ylim()
ax.plot(mm, mm_y, 'b--')
ax.plot(mm - ext_width/2., mm_y, 'g:')
ax.plot(mm + ext_width/2., mm_y, 'g:')
ax.set_title("Cross-dispersion Cut at Pixel=100")

Do the extraction

In [ ]:
x = np.arange(data_lrs_reg.shape[1]) # x in pixels [TBD wavelength instead]
ext1d_boxcar = np.sum(data_lrs_reg[ext_center-ext_width//2:ext_center+ext_width//2, :], axis=0)
# convert from MJy/sr to Jy
ext1d_boxcar *= 1e6*hdu[1].header['PIXAR_SR']

In [ ]:
# plot
fig, ax = plt.subplots(figsize=(6, 6))
gpts = ext1d_boxcar > 0.
ax.plot(x[gpts], ext1d_boxcar[gpts], 'k-', label="boxcar")
ax.set_title("Boxcar 1D extracted spectrum")
ax.set_xlabel("pixel")
ax.set_ylabel("Flux Density [Jy]")

### Wavelength scaled width boxcar

Scaling the width of the extraction aperture with wavelength accounts for the changing diffraction limit with wavelength.

TBD: Handle partial pixels.  Trace the peak versus wavelength to allow for a spectral tilt.

In [ ]:
# TBD
# needs to scale the width linearly with wavelength
# have to handle fractional pixels

## PSF based Extraction

Use the knowledge of the PSF and how it changes with wavelength to provide an improved extration.

In [ ]:
# TBD
# read in a reference file that gives the spatial profile as a function of wavelength (pixel intially)
# fit the spatial profile plus a constant/line for the background for each wavelength
#   fitted scaling factor gives the flux density at that pixel

## Additional Resources
Provide links to appropriate JDox pages for MIRI LRS and JWST pipeline.

- [MIRI LRS](https://jwst-docs.stsci.edu/mid-infrared-instrument/miri-observing-modes/miri-low-resolution-spectroscopy)
- [MIRISim](http://www.stsci.edu/jwst/science-planning/proposal-planning-toolbox/mirisim)
- [JWST pipeline](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline)
- Include a reference for PSF based extraction (Horne?).

## About this notebook

**Author:** Karl Gordon, JWST
**Updated On:** 2020-04-09

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 